Setup necessary modules

In [1]:
%use dataframe
%use kandy

Setup file columns and folders

In [2]:
import java.nio.file.Paths
import java.nio.file.Files
import java.io.BufferedInputStream
import org.apache.commons.csv.CSVFormat
import kotlin.io.path.inputStream

val date by column<Int>("DATE")
val count by column<Int>("count")
val tavg by column<Double>("TAVG")
val tmin by column<Double>("TMIN")
val tmax by column<Double>("TMAX")
val average by column<Double>()
val max by column<Double>()
val min by column<Double>()

val name by column<String>("NAME")
val latitude by column<Double>("LATITUDE")
val longitude by column<Double>("LONGITUDE")
val elevation by column<Double>("ELEVATION")
val snow by column<Double>("SNOW")

//hacky
val projectFolder = Paths.get(System.getProperty("user.dir").replace("/src/main/kotlin/org/data/first/","")).normalize()
println(projectFolder)
val rawDataFolder = projectFolder.resolve("raw-data")
println(rawDataFolder)
val rawDataYearlyDataFolder = rawDataFolder.resolve("yearly-weather")
println(rawDataYearlyDataFolder)
val refinedDataFolder = projectFolder.resolve("refined-data")
println(refinedDataFolder)
val refinedDataYearlyDataFolder = refinedDataFolder.resolve("yearly-data")
println(refinedDataYearlyDataFolder)
val presentableDataFolder = projectFolder.resolve("presentable-data")
println(presentableDataFolder)

val highElevationStart = 2000.0

/Users/jsrnicek/IdeaProjects/weather-data/weather-data
/Users/jsrnicek/IdeaProjects/weather-data/weather-data/raw-data
/Users/jsrnicek/IdeaProjects/weather-data/weather-data/raw-data/yearly-weather
/Users/jsrnicek/IdeaProjects/weather-data/weather-data/refined-data
/Users/jsrnicek/IdeaProjects/weather-data/weather-data/refined-data/yearly-data
/Users/jsrnicek/IdeaProjects/weather-data/weather-data/presentable-data


Count all raw data samples

In [3]:

Files.list(rawDataYearlyDataFolder)
    .map { Files.readAllLines(it).count() }
   .toList()
    .sum()

2658574

Refine data to only the ones that have minimum,maximum and average temparatures available

In [4]:

var counter = 0
Files.list(rawDataYearlyDataFolder)
    .toList()   
    .chunked(1000)
    .forEach { chunk ->
        val outFile = refinedDataYearlyDataFolder.resolve("yearly_data_part_$counter.csv")
        Files.writeString(outFile, chunk.map {
            DataFrame.read(it.toFile())
                .filter { it.containsKey("TAVG") }
                .filter { it.get("TAVG") != null }
                .filter { !tavg.getValue(it).isNaN() }
                .filter { it.containsKey("TMIN") }
                .filter { it.get("TMIN") != null }
                .filter { !tmin.getValue(it).isNaN() }
                .filter { it.containsKey("TMAX") }
                .filter { it.get("TMAX") != null }
                .filter { !tmax.getValue(it).isNaN() }
        }
            .toList()
            .concat()
            .toCsv(CSVFormat.DEFAULT))
        counter += 1
    }


Count data samples after refinement

In [5]:

Files.list(refinedDataYearlyDataFolder)
    .map { Files.readAllLines(it).count() }
    .toList()
    .sum()

869944

Aggregate all files to a single file with count of data samples, min,max and average temparatures

In [16]:


Files.writeString(presentableDataFolder.resolve("basic-temparatures.csv"), Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map { DataFrame.readCSV(BufferedInputStream(it.inputStream())) }
            .toList()
            .concat()
            .groupBy(date)
            .sortBy(date)
            .aggregate {
                count() into count
                mean(tavg) into average
                mean(tmin) into min
                mean(tmax) into max
            }
    }
    .toList()
    .concat()
    .convert(count).with{it.toInt()}
    .groupBy(date)
    .sortBy(date)
    .aggregate {
        sum(count) into count
        mean(average) into average
        mean(min) into min
        mean(max) into max
    }
    .toCsv())

/Users/jsrnicek/IdeaProjects/weather-data/weather-data/presentable-data/basic-temparatures.csv

Plot min/max temparatures

In [17]:

DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("basic-temparatures.csv").toFile().inputStream()))
    .plot {
        line {
            x(date) {
                axis {
                    name = "Year"
                    breaks(format = "{d}")
                }
            }
            y(min) {
                scale = continuous(0.0..20.0)
                axis {
                    name = "Temperatures"
                }
            }
            color(min) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }
        line {
            x(date)
            y(max) { scale = continuous(0.0..20.0) }
            color(max) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }

        layout.size = 1500 to 800
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="mWBjEM"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1763.0,1764.0,1765.0,1766.0,1767.0,1768.0,1769.0,1770.0,1771.0,1772.0,1773.0,1775.0,1776.0,1777.0,1778.0,1779.0,1780.0,1781.0,1782.0,1783.0,1784.0,1785.0,1786.0,1787.0,1788.0,1789.0,1790.0,1791.0,1792.0,1793.0,1794.0,1795.0,1796.0,1797.0,1798.0,1799.0,1800.0,1801.0,1802.0,1803.0,1804.0,1805.0,1806.0,1807.0,1808.0,1809.0,1810.0,1811.0,1812.0,1813.0,1814.0,1815.0,1816.0,1817.0,1818.0,1819.0,1820.0,1821.0,1822.0,1823.0,1824.0,1825.0,1826.0,1827.0,1828.0,1829.0,1830.0,1831.0,1832.0,1833.0,1834.0,1835.0,1836.0,1837.0,1838.0,1839.0,1840.0,1841.0,1842.0,1843.0,1844.0,1845.0,1846.0,1847.0,1848.0,1849.0,1850.0,1851.0,1852.0,1853.0,1854.0,1855.0,1856.0,1857.0,1858.0,1859.0,1860.0,1861.0,1862.0,1863.0,1864.0,1865.0,1866.0,1867.0,1868.0,1869.0,1870.0,1871.0,1872.0,1873.0,1874.0,1875.0,1876.0,1877.0,1878.0,1879.0,1880.0,1881.0,1882.0,1883.0,1884.0,1885.0,1886.0,1887.0,1888.0,1889.0,1890.0,1891.0,1892.0,1893.0,1894.0,1895.0,1896.0,1897.0,1898.0,1899.0,1900.0,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0],
"min":[9.98,10.6,10.49,10.14,9.14,9.85,10.67,10.63,10.78,12.15,10.54,9.26,7.865,7.76,6.65,6.82,5.34,9.095,5.77,8.965,7.609999999999999,7.41,7.16,9.105,6.44,8.530000000000001,8.915,8.02,8.955,9.26,8.49,8.355,8.74,9.290000000000001,8.84,6.970000000000001,6.99,9.385,9.145,6.13,6.54,7.109999999999999,9.335,7.41,7.55,8.43,6.03,9.565,7.185,6.32,5.08,6.646666666666667,6.150000000000001,6.82,7.816666666666666,7.813333333333333,6.9433333333333325,8.31,9.325,6.986666666666667,6.8575,7.25,7.15,6.906666666666666,7.5525,5.21,6.41,6.380000000000001,6.175,6.282,7.142,5.984,6.048,5.55,4.51,6.042,4.894,5.67,5.194000000000001,5.865,5.189166666666668,5.168000000000001,6.040000000000001,5.253333333333333,5.586666666666666,5.423333333333333,5.187500000000001,5.484166666666667,6.003333333333334,5.043333333333333,5.3500000000000005,4.615,6.126428571428571,6.367500000000001,5.624375,6.815555555555555,6.279375,5.8375,5.980499999999999,7.453571428571428,4.491666666666666,5.306111111111111,5.861000000000001,4.37925,6.159363636363637,3.749538461538462,4.3412500000000005,4.370288888888889,5.3289687500000005,4.217415178571429,4.609774509803921,3.4397586367880484,4.224865520282188,4.722109126984126,5.398341490299824,3.8106052604802607,4.417242724867724,4.142311427769761,3.8463334258334263,2.4129045214045215,2.7263340748757416,2.1386072153155484,2.3362830705593387,2.3464267013850346,2.467287155502068,3.400919090305119,3.050199140365807,3.1829285017552946,3.210358086240422,3.5974678625548293,4.602027456979906,3.8334408199750474,4.74424019089788,4.610441223706761,4.829648660322523,4.184084680910571,4.96472210140924,4.402652562616303,4.483305090184555,4.5328047034291306,3.90023174722728,4.3794664444519125,5.055181118009184,4.6704568234018495,5.029104108090305,4.5963492

Plot weather stations

In [18]:

DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("basic-temparatures.csv").toFile().inputStream()))
    .plot {
        line {
            x(date) {
                axis {
                    name = "Year"
                    breaks(format = "{d}")
                }
            }
            y(count) {
                axis {
                    name = "Weather stations"
                }
            }
            color(count) {
                scale = continuous(Color.RED..Color.GREEN)
                legend {
                    name = "Weather stations"
                }
            }
        }

        layout.size = 1500 to 800
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="qZEeYq"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1763.0,1764.0,1765.0,1766.0,1767.0,1768.0,1769.0,1770.0,1771.0,1772.0,1773.0,1775.0,1776.0,1777.0,1778.0,1779.0,1780.0,1781.0,1782.0,1783.0,1784.0,1785.0,1786.0,1787.0,1788.0,1789.0,1790.0,1791.0,1792.0,1793.0,1794.0,1795.0,1796.0,1797.0,1798.0,1799.0,1800.0,1801.0,1802.0,1803.0,1804.0,1805.0,1806.0,1807.0,1808.0,1809.0,1810.0,1811.0,1812.0,1813.0,1814.0,1815.0,1816.0,1817.0,1818.0,1819.0,1820.0,1821.0,1822.0,1823.0,1824.0,1825.0,1826.0,1827.0,1828.0,1829.0,1830.0,1831.0,1832.0,1833.0,1834.0,1835.0,1836.0,1837.0,1838.0,1839.0,1840.0,1841.0,1842.0,1843.0,1844.0,1845.0,1846.0,1847.0,1848.0,1849.0,1850.0,1851.0,1852.0,1853.0,1854.0,1855.0,1856.0,1857.0,1858.0,1859.0,1860.0,1861.0,1862.0,1863.0,1864.0,1865.0,1866.0,1867.0,1868.0,1869.0,1870.0,1871.0,1872.0,1873.0,1874.0,1875.0,1876.0,1877.0,1878.0,1879.0,1880.0,1881.0,1882.0,1883.0,1884.0,1885.0,1886.0,1887.0,1888.0,1889.0,1890.0,1891.0,1892.0,1893.0,1894.0,1895.0,1896.0,1897.0,1898.0,1899.0,1900.0,1901.0,1902.0,1903.0,1904.0,1905.0,1906.0,1907.0,1908.0,1909.0,1910.0,1911.0,1912.0,1913.0,1914.0,1915.0,1916.0,1917.0,1918.0,1919.0,1920.0,1921.0,1922.0,1923.0,1924.0,1925.0,1926.0,1927.0,1928.0,1929.0,1930.0,1931.0,1932.0,1933.0,1934.0,1935.0,1936.0,1937.0,1938.0,1939.0,1940.0,1941.0,1942.0,1943.0,1944.0,1945.0,1946.0,1947.0,1948.0,1949.0,1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0],
"count":[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,3.0,3.0,1.0,3.0,3.0,3.0,2.0,2.0,3.0,4.0,4.0,4.0,3.0,4.0,4.0,4.0,2.0,4.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,6.0,6.0,5.0,7.0,6.0,6.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,9.0,10.0,10.0,12.0,11.0,12.0,12.0,10.0,13.0,13.0,19.0,20.0,20.0,25.0,26.0,31.0,41.0,50.0,61.0,74.0,79.0,89.0,103.0,116.0,118.0,128.0,146.0,150.0,173.0,178.0,200.0,214.0,210.0,232.0,251.0,263.0,289.0,648.0,866.0,998.0,1076.0,1214.0,1328.0,1390.0,1444.0,1268.0,1415.0,1549.0,1620.0,1742.0,1753.0,1747.0,1832.0,1945.0,2044.0,2142.0,2215.0,2290.0,2387.0,2459.0,2431.0,2468.0,2372.0,2368.0,2379.0,2502.0,2523.0,2578.0,2638.0,2729.0,2783.0,2809.0,2916.0,2960.0,3136.0,3298.0,3349.0,3458.0,3481.0,3500.0,3600.0,3702.0,3889.0,4082.0,4160.0,4211.0,4291.0,4314.0,4538.0,4600.0,4695.0,4770.0,4800.0,5511.0,5800.0,6464.0,6808.0,7088.0,7307.0,7537.0,7680.0,7970.0,8196.0,8326.0,8622.0,9207.0,9510.0,9868.0,9830.0,10206.0,10465.0,10630.0,10372.0,10328.0,10828.0,10881.0,10788.0,10437.0,10704.0,10738.0,10643.0,10681.0,10646.0,10353.0,10351.0,10661.0,10218.0,10415.0,10100.0,9722.0,9656.0,9794.0,9741.0,9832.0,9667.0,9953.0,9725.0,9353.0,9709.0,9936.0,10072.0,10129.0,10089.0,10156.0,10202.0,10270.0,10328.0,10474.0,10420.0,10596.0,10643.0,10439.0,10240.0,10543.0,10784.0,11039.0,10326.0,9366.0,9384.0,9372.0,9117.0,8888.0,8801.0,8344.0,8523.0,8435.0,8231.0,7998.0]
},
"ggsize":{
"width":1500.0,
"height":800.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Year",
"format":"{d}",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Weather stations",
"limits":[null,null]
},{
"aesthetic":"color"

Put data for all stations into GPX format so we can feed it int omap

In [19]:
data class GpxEntry(val name: String, val latitude: Double, val longitude: Double) {
    override fun toString(): String {
        return """<wpt lat="${latitude}" lon="${longitude}">
		            <name>${name}</name>
	              </wpt>"""
    }
}

data class GpxTemplate(val points: List<GpxEntry>) {
    override fun toString(): String {
        return """<?xml version="1.0" encoding="UTF-8"?>
                    <gpx version="1.1" xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd">
	                    ${points.map { it.toString() }.joinToString("\n")}
                    </gpx>"""
    }
}

In [20]:


val points = Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map { DataFrame.readCSV(BufferedInputStream(it.inputStream())) }
            .toList()
            .concat()
            .groupBy(name, latitude, longitude)
            .count()
    }
    .toList()
    .concat()
    .groupBy(name, latitude, longitude)
    .count()
    .map {
        GpxEntry(
            name = it.getValue(name), latitude = it.getValue(latitude), longitude = it.getValue(longitude)
        )
    }
    .toList()

Files.writeString(presentableDataFolder.resolve("gpx-visualization.gpx"),GpxTemplate(points).toString())


/Users/jsrnicek/IdeaProjects/weather-data/weather-data/presentable-data/gpx-visualization.gpx

In [21]:
Files.writeString(presentableDataFolder.resolve("duplicate-values.csv"),Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map {
            DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                .filter { date.getValue(it) >= 1900 }
        }
            .toList()
            .concat()
            .groupBy(name,date)
            .aggregate {
                count() into count
            }
    }
    .toList()
    .concat()
    .convert(count).with { it.toInt() }
    .groupBy(name,date)
    .aggregate {
        sum(count) into count
    }
    .filter { count.getValue(it) > 1 }//duplicates
    .sortByDesc(date)
    .toCsv())

class java.lang.Double cannot be cast to class java.lang.Integer (java.lang.Double and java.lang.Integer are in module java.base of loader 'bootstrap')
java.lang.ClassCastException: class java.lang.Double cannot be cast to class java.lang.Integer (java.lang.Double and java.lang.Integer are in module java.base of loader 'bootstrap')
	at Line_28_jupyter$res28$3.invoke(Line_28.jupyter.kts:28)
	at Line_28_jupyter$res28$3.invoke(Line_28.jupyter.kts:20)
	at org.jetbrains.kotlinx.dataframe.impl.GroupByImplKt.aggregateGroupBy(GroupByImpl.kt:87)
	at org.jetbrains.kotlinx.dataframe.impl.GroupByImpl.aggregate(GroupByImpl.kt:58)
	at Line_28_jupyter.<init>(Line_28.jupyter.kts:20)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:502)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:486)
	at kotlin.script.experimental.jvm.Bas

In [22]:
DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("duplicate-values.csv").inputStream()))
    .take(10)

NAME,DATE,count
SAVANNAH HILTON HEAD INTERNATIONAL AI...,2023,2.000000
"MARYBOROUGH, AS",2023,2.000000
"TERRE HAUTE HULMAN REGIONAL AIRPORT, ...",2023,2.000000
"APALACHICOLA AIRPORT, FL US",2023,2.000000
"VICTORIA REGIONAL AIRPORT, TX US",2023,2.000000
"SAN JOSE, CA US",2023,2.000000
"CORNER BROOK, NL CA",2023,2.000000
"YORKTON, SK CA",2023,2.000000
"BAIE COMEAU, QC CA",2023,2.000000
"TROIS RIVIERES, QC CA",2023,2.000000


In [29]:
val limitYear = 1950
val expectedValueCount = 2023 - limitYear + 1

val potentialCandidates = Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map {
            DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                .filter { date.getValue(it) >= limitYear }
        }
            .toList()
            .concat()
            .groupBy(name)
            .aggregate {
                count() into count
            }
    }
    .toList()
    .concat()
    .convert(count).with { it.toInt() }
    .groupBy(name)
    .aggregate {
        sum(count) into count
    }
    .filter { count.getValue(it) >= expectedValueCount }//duplicates
    .sortByDesc(count)
    .select(name)
    .convert(name).with { it.toString() }
    .map { name.getValue(it) }

val weatherStations = Files.list(refinedDataYearlyDataFolder)
    .toList()
    .chunked(5)
    .map { chunk ->
        chunk.map {
            DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                .filter { potentialCandidates.contains(name.getValue(it)) }
                .filter { date.getValue(it) >= expectedValueCount }
        }
            .toList()
            .concat()
    }
    .toList()
    .concat()
    .distinctBy(name, date)
    .groupBy(name)
    .sortBy(name, date)
    .aggregate {
        count() into count
    }
    .convert(count).with { it.toInt() }
    .filter { count.getValue(it) == expectedValueCount }
    .select(name)
    .convert(name).with { it.toString() }
    .map { name.getValue(it) }


Files.writeString(refinedDataYearlyDataFolder.resolve("solid-data-since-$limitYear.csv"),
    Files.list(refinedDataYearlyDataFolder)
        .toList()
        .chunked(5)
        .map { chunk ->
            chunk.map {
                DataFrame.readCSV(BufferedInputStream(it.inputStream()))
                    .filter { weatherStations.contains(name.getValue(it)) }
                    .filter { date.getValue(it) >= limitYear }
            }
                .toList()
                .concat()
        }
        .toList()
        .concat()
        .toCsv())

/Users/jsrnicek/IdeaProjects/weather-data/weather-data/refined-data/yearly-data/solid-data-since-1950.csv

In [30]:
Files.writeString(presentableDataFolder.resolve("yearly-temparatures-since-$limitYear.csv"),DataFrame.readCSV(BufferedInputStream(
    refinedDataYearlyDataFolder.resolve("solid-data-since-$limitYear.csv").inputStream()
))
    .groupBy(date)
    .aggregate {
        mean(tmin) into min
        mean(tmax) into max
        mean(tavg) into average
    }
    .toCsv())

/Users/jsrnicek/IdeaProjects/weather-data/weather-data/presentable-data/yearly-temparatures-since-1950.csv

Plot solid data since 1900

In [31]:
DataFrame.readCSV(BufferedInputStream(presentableDataFolder.resolve("yearly-temparatures-since-$limitYear.csv").toFile().inputStream()))
    .plot {
        line {
            x(date) {
                axis {
                    name = "Year"
                    breaks(format = "{d}")
                }
            }
            y(min) {
                scale = continuous(0.0..20.0)
                axis {
                    name = "Temperatures"
                }
            }
            color(min) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }
        line {
            x(date)
            y(max) { scale = continuous(0.0..20.0) }
            color(max) {
                scale = continuous(Color.BLUE..Color.RED)
                legend {
                    name = "Temparature in Celsius"
                }
            }
        }

        layout.size = 1500 to 800
    }

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="NpNUKT"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"DATE":[1950.0,1951.0,1952.0,1953.0,1954.0,1955.0,1956.0,1957.0,1958.0,1959.0,1960.0,1961.0,1962.0,1963.0,1964.0,1965.0,1966.0,1967.0,1968.0,1969.0,1970.0,1971.0,1972.0,1973.0,1974.0,1975.0,1976.0,1977.0,1978.0,1979.0,1980.0,1981.0,1982.0,1983.0,1984.0,1985.0,1986.0,1987.0,1988.0,1989.0,1990.0,1991.0,1992.0,1993.0,1994.0,1995.0,1996.0,1997.0,1998.0,1999.0,2000.0,2001.0,2002.0,2003.0,2004.0,2005.0,2006.0,2007.0,2008.0,2009.0,2010.0,2011.0,2012.0,2013.0,2014.0,2015.0,2016.0,2017.0,2018.0,2019.0,2020.0,2021.0,2022.0,2023.0],
"min":[2.45,2.7925,3.7475,4.11,4.112500000000001,3.2925000000000004,3.54,3.95,3.6975,3.7575,3.3674999999999997,3.5175,3.585,3.43,3.7649999999999997,3.9425000000000003,3.3025,3.6274999999999995,3.8575,3.6950000000000003,3.85,3.8575,3.4174999999999995,4.8325,4.0925,4.1075,3.1550000000000002,4.3125,3.3625,3.3975,4.2275,4.35,3.6625,4.4750000000000005,4.2075,3.16,4.4325,5.03,3.4650000000000003,3.936666666666667,4.6025,6.140000000000001,4.572500000000001,3.9275,4.215,4.125,3.1399999999999997,3.44,5.760000000000001,3.8260000000000005,3.228,3.726,4.7175,4.085,4.2139999999999995,4.406,4.642,4.1259999999999994,3.226,4.1925,4.825,4.742500000000001,5.62,4.1775,3.8,5.4775,5.8775,5.2975,4.907500000000001,4.5675,4.748,4.779999999999999,4.38,5.795],
"max":[13.01,13.2525,14.7825,15.225000000000001,14.474999999999998,13.934999999999999,14.465,13.96,14.07,13.8325,13.465,13.847499999999998,13.7375,14.35,14.27,13.91,13.615,13.7525,13.675,13.382499999999999,13.540000000000003,13.85,13.0575,14.115,13.8625,13.629999999999999,14.2025,14.149999999999999,13.2325,13.1575,14.075,14.1825,13.3275,13.712499999999999,13.870000000000001,13.3075,14.61,15.8825,15.1875,15.933333333333332,15.43,17.046666666666667,14.602500000000001,13.35,14.7025,14.382500000000002,13.375,12.624,15.780000000000001,13.876,12.978000000000003,13.422,14.717500000000001,14.435,13.797999999999998,14.298000000000002,14.486,13.652000000000001,12.594,13.875,14.6075,14.305,16.32,13.899999999999999,13.5275,15.34,15.5075,15.205,14.584999999999999,14.155,13.884,14.080000000000002,14.745000000000001,15.965]
},
"ggsize":{
"width":1500.0,
"height":800.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"Year",
"format":"{d}",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"Temperatures",
"limits":[0.0,20.0]
},{
"aesthetic":"color",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#5470c6",
"name":"Temparature in Celsius",
"limits":[null,null]
},{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[0.0,20.0]
},{
"aesthetic":"color",
"scale_mapper_kind":"color_gradient",
"high":"#ee6666",
"low":"#5470c6",
"name":"Temparature in Celsius",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"DATE",
"y":"min",
"color":"min"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
},{
"mapping":{
"x":"DATE",
"y":"max",
"color":"max"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("NpNUKT");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1950 
 
 
 
 
 
 
 
 
 1955 
 
 
 
 
 
 
 
 
 1960 
 
 
 
 
 
 
 
 
 1965 
 
 
 
 
 
 
 
 
 1970 
 
 
 
 
 
 
 
 
 1975 
 
 
 
 
 
 
 
 
 1980 
 
 
 
 
 
 
 
 
 1985 
 
 
 
 
 
 
 
 
 1990 
 
 
 
 
 
 
 
 
 1995 
 
 
 
 
 
 
 
 
 2000 
 
 
 
 
 
 
 
 
 2005 
 
 
 
 
 
 
 
 
 2010 
 
 
 
 
 
 
 
 
 2015 
 
 
 
 
 
 
 
 
 2020 
 
 
 
 
 
 
 
 
 2025 
 